In [2]:
import pandas as pd
from pyproj import Transformer
import os
pd.set_option('display.max_rows', 500)

In [3]:
path_to_file = os.path.join('data', 'ContinuousAirQuality.csv')
df = pd.read_csv(path_to_file)
df.head()

/var/folders/_k/0mgp9cxj72dcxh4819b8hd740000gn/T/ipykernel_18946/681335085.py:2: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_to_file)


,SITE_ID,LOCATION,EASTING,NORTHING,DATE_TIME,NO,NOX,NO2,PM2_5,PM10
0,188,AURN Bristol Centre,359406,173299,01/01/1993 00:00,NaN,NaN,NaN,NaN,NaN
1,188,AURN Bristol Centre,359406,173299,01/01/1993 01:00,NaN,NaN,NaN,NaN,NaN
2,188,AURN Bristol Centre,359406,173299,01/01/1993 02:00,NaN,NaN,NaN,NaN,NaN
3,188,AURN Bristol Centre,359406,173299,01/01/1993 03:00,NaN,NaN,NaN,NaN,NaN
4,188,AURN Bristol Centre,359406,173299,01/01/1993 04:00,NaN,NaN,NaN,NaN,NaN


In [4]:
print(f"Shape: {df.shape}")

Shape: (1607930, 10)


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1607930 entries, 0 to 1607929
Data columns (total 10 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   SITE_ID    1607930 non-null  int64  
 1   LOCATION   1607930 non-null  object 
 2   EASTING    1607930 non-null  int64  
 3   NORTHING   1607930 non-null  int64  
 4   DATE_TIME  1607930 non-null  object 
 5   NO         1478294 non-null  float64
 6   NOX        1471958 non-null  float64
 7   NO2        1469173 non-null  float64
 8   PM2_5      154006 non-null   object 
 9   PM10       358192 non-null   object 
dtypes: float64(3), int64(3), object(4)
memory usage: 122.7+ MB


In [6]:
df[['PM2_5', 'PM10']] = df[['PM2_5', 'PM10']].apply(pd.to_numeric, errors='coerce')

In [7]:
df.describe()

,SITE_ID,EASTING,NORTHING,NO,NOX,NO2,PM2_5,PM10
count,1.607930e+06,1.607930e+06,1.607930e+06,1.478294e+06,1.471958e+06,1.469173e+06,150898.000000,357573.000000
mean,3.194403e+02,3.597799e+05,1.727147e+05,4.464865e+01,1.095538e+02,4.181721e+01,11.380196,21.603099
std,1.219205e+02,1.853576e+03,1.783203e+03,7.167152e+01,1.347996e+02,3.082562e+01,10.516486,15.414169
min,1.880000e+02,3.522750e+05,1.700240e+05,-1.771000e+01,-3.108000e+01,-4.160000e+00,-6.000000,-5.000000
25%,2.060000e+02,3.586670e+05,1.715660e+05,6.000000e+00,3.100000e+01,1.950000e+01,5.500000,11.600000
50%,2.700000e+02,3.595550e+05,1.731080e+05,1.900000e+01,6.500000e+01,3.475000e+01,8.443200,18.000000
75%,4.520000e+02,3.610000e+05,1.736830e+05,5.050000e+01,1.320000e+02,5.575000e+01,13.900000,27.000000
max,6.720000e+02,3.629260e+05,1.770080e+05,1.231250e+03,2.164250e+03,5.765000e+02,380.604400,257.009000


In [8]:
df['date'] = df['DATE_TIME'].str.split(' ').str[0]
df['time'] = df['DATE_TIME'].str.split(' ').str[1]

mask = df['time'].str.startswith('24')

df.loc[mask, 'time'] = '00:00:00'

df['DATE_TIME'] = pd.to_datetime(df['date'] + ' ' + df['time'], format='mixed')

df.loc[mask, 'DATE_TIME'] = df.loc[mask, 'DATE_TIME'] + pd.Timedelta(days=1)

df = df.drop(columns=['date', 'time'])

In [9]:
transformer = Transformer.from_crs("epsg:27700", "epsg:4326")

df['Latitude'], df['Longitude'] = transformer.transform(df['EASTING'].values, df['NORTHING'].values)

df = df.drop(columns=['EASTING', 'NORTHING'])

df.head()

,SITE_ID,LOCATION,DATE_TIME,NO,NOX,NO2,PM2_5,PM10,Latitude,Longitude
0,188,AURN Bristol Centre,1993-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,51.457204,-2.585649
1,188,AURN Bristol Centre,1993-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,51.457204,-2.585649
2,188,AURN Bristol Centre,1993-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,51.457204,-2.585649
3,188,AURN Bristol Centre,1993-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,51.457204,-2.585649
4,188,AURN Bristol Centre,1993-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,51.457204,-2.585649


In [10]:
file_path = os.path.join('data', 'processed_data.csv')
df.to_csv(file_path, index=False)